In [2]:
from IPython.display import clear_output, display
import time
import matplotlib.pylab as plt
import numpy as np
import seaborn as sns; sns.set()
%matplotlib inline
from os.path import join

import keras
from keras import objectives
from keras.callbacks import EarlyStopping
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Multiply, Add
from keras.optimizers import Adam, Nadam
from keras.models import load_model

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from random import shuffle
import pandas as pd
import random

#Keras build
from keras import backend as K
from keras.objectives import binary_crossentropy #objs or losses
from keras.models import Model
from keras.layers import Input, Dense, Lambda, Layer
from keras.layers.core import Dense, Activation, Flatten, RepeatVector
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import GRU
from keras.layers.convolutional import Convolution1D

#chem
from rdkit import Chem
from rdkit.Chem import AllChem as Chem
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import DataStructs
from rdkit.Chem import Draw
from rdkit.Chem import MACCSkeys
import salty
import gains as genetic

#cation data
cations = pd.read_csv('../data/cations.csv')
cations = cations['smiles_string']
salts = pd.read_csv('../data/salts.csv')
salts = salts['smiles_string']
categories = pd.read_csv('../data/categories.csv')
categories = categories['category']
coldic = pd.read_csv('../data/coldic.csv')
coldic = coldic.to_dict(orient='records')[0]
salt_coldic = pd.read_csv('../data/salt_coldic.csv')
salt_coldic = salt_coldic.to_dict(orient='records')[0]
salt_categories = pd.read_csv('../data/salt_categories.csv')
salt_categories = salt_categories['category']
density_coldic = pd.read_csv('../data/density_coldic.csv')
density_coldic = density_coldic.to_dict(orient='records')[0]
density_categories = pd.read_csv('../data/density_categories.csv')
density_categories = density_categories['category']

#supporting functions
import sys
sys.path.insert(0, '../')
from scripts import *

#training array info
smile_max_length = 105
f = open("../data/salt_char_to_index.json","r")
ani_char_to_index = json.loads(f.read())
ani_char_set = set(ani_char_to_index.keys())
ani_char_list = list(ani_char_to_index.keys())
ani_chars_in_dict = len(ani_char_list)
ani_index_to_char = dict((i, c) for i, c in enumerate(ani_char_list))

#training array info
import json
f = open("../data/gdb_char_to_index.json","r")
cat_char_to_index = json.loads(f.read())
cat_char_set = set(cat_char_to_index.keys())
cat_char_list = list(cat_char_to_index.keys())
cat_chars_in_dict = len(cat_char_list)
cat_index_to_char = dict((i, c) for i, c in enumerate(cat_char_list))

char_to_index = ani_char_to_index
char_set = ani_char_set
char_list = ani_char_list
chars_in_dict = ani_chars_in_dict
index_to_char = ani_index_to_char

In [7]:
gen1vae = MoleculeVAE()
gen1vae.create(char_set, qspr=True, mol_inputs=2, weights_file='../models/gen1_2mol_1mil_GDB17_mix_pure_5.h5')
gen2vae = TwoMoleculeOneLatentVAE()
gen2vae.create(char_set, qspr=True, weights_file='../models/gen2_2mol_1mil_GDB17_mix_pure_5.h5')
gen3vae = TwoMoleculeVAE()
gen3vae.create(char_set, char_set, qspr=True, weights_file='../models/gen3_2mol_1mil_GDB17_mix_pure_5.h5')

In [3]:
salts = pd.read_csv('../data/salts.csv')
salts = salts['smiles_string']

In [8]:
df_gen3 = generate_solvent_vae(vae=gen3vae, 
                     char_to_index=char_to_index,
                     smile_max_length=62,
                     salts=salts, 
                     model_ID=None, 
                     target=None, 
                     qspr=False, 
                     find=100, 
                     optimalCutOff=None, 
                     greaterThanCutOff=False,
                     md_model=None,
                     verbose=1)

ani seed                             CCCCO[P@@H](=O)[O-]
attempts                                            2597
candidate                          Cc1ccc[n+](CCCCCCO)c1
cat seed                           CCCCCCC[n+]1cc(C)ccc1
salt           Cc1ccc[n+](CCCCCCO)c1.CCCCO[P@@H](=O)[O-]
temperature                                      1.22805
Name: 99, dtype: object
100/100 found


In [9]:
df_gen1 = generate_solvent_vae(vae=gen1vae, 
                     char_to_index=char_to_index,
                     smile_max_length=62,
                     salts=salts, 
                     model_ID=None, 
                     target=None, 
                     qspr=False, 
                     find=100, 
                     optimalCutOff=None, 
                     greaterThanCutOff=False,
                     md_model=None,
                     verbose=1)

ani seed                                N#C[B-](C#N)(C#N)C#N
attempts                                                4073
candidate                           CCCCCCCCCCCCCCCCCCCCCCCC
cat seed                             c1c[n+](cn1CCCCCCCCCC)C
salt           CCCCCCCCCCCCCCCCCCCCCCCC.N#C[B-](C#N)(C#N)C#N
temperature                                              0.1
Name: 99, dtype: object
100/100 found


In [10]:
df_gen2 = generate_solvent_vae(vae=gen2vae, 
                     char_to_index=char_to_index,
                     smile_max_length=62,
                     salts=salts, 
                     model_ID=None, 
                     target=None, 
                     qspr=False, 
                     find=100, 
                     optimalCutOff=None, 
                     greaterThanCutOff=False,
                     md_model=None,
                     verbose=1)

ani seed                C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F
attempts                                                   13968
candidate                                 CCCCCCCCC1CCCCCCCCCCC1
cat seed                                c1cn(c([n+]1C))CCCCC(C)C
salt           CCCCCCCCC1CCCCCCCCCCC1.C(F)(F)(F)S(=O)(=O)[N-]...
temperature                                             0.429939
Name: 99, dtype: object
100/100 found


In [11]:
df = pd.read_csv('../data/gen3vae_no_target_2597_function_calls', index_col=0)

In [12]:
df

,ani seed,attempts,candidate,cat seed,salt,temperature
0,c1c(cn[n-]1)[N+](=O)[O-],3,CCCC[P+](CCCC)(CCCC)CCCCO,CCCC[P+](CCCC)(CCCC)CCCC,CCCC[P+](CCCC)(CCCC)CCCCO.c1c(cn[n-]1)[N+](=O)...,1.846531
1,C(#N)[N-]C#N,25,C[n+]1ccn(CCCC#N)c1,CCCCCCn1cc[n+](c1)C,C[n+]1ccn(CCCC#N)c1.C(#N)[N-]C#N,0.209841
2,[B-](F)(F)(F)F,47,C[n+]1ccn(CCCC#N)c1,CCCCCCn1cc[n+](c1)C,C[n+]1ccn(CCCC#N)c1.[B-](F)(F)(F)F,1.164818
3,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,62,C[n+]1ccn(CCCCCO)c1,CCCCCCn1cc[n+](c1)C,C[n+]1ccn(CCCCCO)c1.C(F)(F)(F)S(=O)(=O)[N-]S(=...,1.089706
4,CC1=NC=C(N1)[N+](=O)[O-],67,C#CCCCn1cc[n+](C)c1,CCCCCCn1cc[n+](c1)C,C#CCCCn1cc[n+](C)c1.CC1=NC=C(N1)[N+](=O)[O-],0.150706
5,C(F)(F)(F)S(=O)(=O)[O-],183,CCCCCCn1cc[n+](C)c1,N#CCCCn1cc[n+](c1)C,CCCCCCn1cc[n+](C)c1.C(F)(F)(F)S(=O)(=O)[O-],0.768544
6,C(=C(C#N)C#N)=[N-],190,C[n+]1ccn(CCCCNN)c1,CCCCCCn1cc[n+](c1)C,C[n+]1ccn(CCCCNN)c1.C(=C(C#N)C#N)=[N-],1.287164
7,C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O)C(F)(F)F,192,CCC[N+](C)(C)C,OCC[N+](C)(C)C,CCC[N+](C)(C)C.C(F)(F)(F)S(=O)(=O)[N-]S(=O)(=O...,0.738129
8,N#C[B-](C#N)(C#N)C#N,197,CCCC[P+]CC,CCCC[P+](CCCC)(CCCC)CCCC,CCCC[P+]CC.N#C[B-](C#N)(C#N)C#N,1.778158
9,CCCCC(CC)COC(=O)CC(C(=O)OCC(CC)CCCC)S(=O)(=O)[O-],206,C[n+]1ccn(CCCCCN)c1,CCCCCCn1cc[n+](c1)C,C[n+]1ccn(CCCCCN)c1.CCCCC(CC)COC(=O)CC(C(=O)OC...,0.726118


In [11]:
gen1_results = pd.read_csv('../data/function_call_comparisons/gen1vae_no_target_4073_function_calls', index_col=0)
gen2_results = pd.read_csv('../data/function_call_comparisons/gen2vae_no_target_13968_function_calls', index_col=0)
gen3_results = pd.read_csv('../data/function_call_comparisons/gen3vae_no_target_2597_function_calls', index_col=0)b
display(gen1_results.head())
display(gen2_results.head())
display(gen3_results.head())